In [67]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

random_state = 123

In [68]:
df_full = pd.read_csv("combined_regular_clean_with_ssurgo_variables.csv")
df_full.shape

(14619, 29)

# Filter out bad longitude

In [69]:
# remove longitude > -50 (bad datapoints)

df_full = df_full[df_full.longitude < -50]
df_full.shape # 6 records removed

(14613, 29)

# Filter out bad records

- any records where the cwa_determination is contrary to expectations? That is, where none of cwa1 etc are 1 but the cwa_determination value is 1


In [70]:
# any records where the cwa_determination is contrary to expectations? That is, where none of cwa1 etc
# are 1 but the cwa_determination value is 1
good_records = (df_full.apply(lambda x: 
               (np.sum(x.cwa1 + x.cwa2 + x.cwa3 + x.cwa4 + x.cwa5 + 
                       x.cwa6 + x.cwa7 + x.cwa8 + x.cwa9) > 0) * 1 
               == x.cwa_determination, 
               axis=1))

print("%good records = {}%".format(round(np.mean(good_records) * 100)))
print("There are {}% bad records".format(100-round(np.mean(good_records) * 100)))
print(df_full.shape)
# peek at not good records to verify code
df_full[~good_records].head(3).iloc[:,10:30]


%good records = 98%
There are 2% bad records
(14613, 29)


,cwa_determination,rha1,rha2,cwa1,cwa2,cwa3,cwa4,cwa5,cwa6,cwa7,cwa8,cwa9,potential_wetland,index,Index,mukey,hydclprs,aws025wta,drclassdcd
36,1,0,0,0,0,0,0,0,0,0,0,0,1,37,37,292681.0,0.0,4.48,Moderately well drained
50,1,0,0,0,0,0,0,0,0,0,0,0,1,51,51,292995.0,96.0,4.65,Very poorly drained
56,1,0,0,0,0,0,0,0,0,0,0,0,1,57,57,292980.0,8.0,4.67,Somewhat poorly drained


In [71]:
# Retain only the good records
df_full = df_full[good_records]
df_full.shape # 291 records removed

(14322, 29)

# Drop St. Louis entirely (it has only 6 records, and is not being split in 70/15/15, and causing other issues)

In [72]:
df_full = df_full[df_full.district != "St. Louis"]

In [73]:
df_full.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'jurisdiction_type', 'da_number',
       'district', 'project_name', 'longitude', 'latitude',
       'date_issued_or_denied', 'rha_determination', 'cwa_determination',
       'rha1', 'rha2', 'cwa1', 'cwa2', 'cwa3', 'cwa4', 'cwa5', 'cwa6', 'cwa7',
       'cwa8', 'cwa9', 'potential_wetland', 'index', 'Index', 'mukey',
       'hydclprs', 'aws025wta', 'drclassdcd'],
      dtype='object')

# Split data into Train, Test1 and Test2

In [74]:
# verify if things are indeed working as expected

mygroup = df_full.groupby("district")

for group in mygroup:
    print(group[0])
    print("Original 0's and 1's")
    data_count_1 = group[1]["cwa_determination"][group[1]["cwa_determination"]==1].shape[0]
    data_count_0 = group[1]["cwa_determination"][group[1]["cwa_determination"]==0].shape[0]    
    print(data_count_0, data_count_1)
    print()
    
    train, test = (train_test_split(group[1],
                                    test_size=0.15,
                                    random_state = random_state, 
                                    stratify=group[1]["cwa_determination"]))
    
    print("train total records", train.shape[0])
    train_count_1 = train["cwa_determination"][train["cwa_determination"]==1].shape[0]
    train_count_0 = train["cwa_determination"][train["cwa_determination"]==0].shape[0]    
    print("train 0's and 1's:")
    print(train_count_0, train_count_1)
    
    print()
    
    print("test total records", test.shape[0])
    print("test 0's and 1's:")
    test_count_1 = test["cwa_determination"][test["cwa_determination"]==1].shape[0]
    test_count_0 = test["cwa_determination"][test["cwa_determination"]==0].shape[0]    
    print(test_count_0, test_count_1)    
    
    print("======================")
    

Alaska
Original 0's and 1's
346 101

train total records 379
train 0's and 1's:
293 86

test total records 68
test 0's and 1's:
53 15
Albuquerque
Original 0's and 1's
121 7

train total records 108
train 0's and 1's:
102 6

test total records 20
test 0's and 1's:
19 1
Baltimore
Original 0's and 1's
50 87

train total records 116
train 0's and 1's:
42 74

test total records 21
test 0's and 1's:
8 13
Buffalo
Original 0's and 1's
119 158

train total records 235
train 0's and 1's:
101 134

test total records 42
test 0's and 1's:
18 24
Charleston
Original 0's and 1's
1307 750

train total records 1748
train 0's and 1's:
1111 637

test total records 309
test 0's and 1's:
196 113
Chicago
Original 0's and 1's
236 332

train total records 482
train 0's and 1's:
200 282

test total records 86
test 0's and 1's:
36 50
Detroit
Original 0's and 1's
93 81

train total records 147
train 0's and 1's:
79 68

test total records 27
test 0's and 1's:
14 13
Fort Worth
Original 0's and 1's
6 12

train total

In [75]:
# Implement the modified stratification

df_train = pd.DataFrame()
df_test1 = pd.DataFrame()
df_test2 = pd.DataFrame()

df_train_ = pd.DataFrame()
df_test1_ = pd.DataFrame()
df_test2_ = pd.DataFrame()

for group in df_full.groupby("district"):
    try:
        df_train_temp_, df_test2_ = (train_test_split(group[1], 
                                                 test_size=0.15, 
                                                 random_state = random_state, 
                                                 stratify=group[1]["cwa_determination"])) # 15% test

        df_train_, df_test1_ = (train_test_split(df_train_temp_, 
                                                 test_size=0.17647, 
                                                 random_state = random_state, 
                                                 stratify=df_train_temp_["cwa_determination"])) # 70% train, 15% dev
    except Exception as e:
        print(group[0], e)
    df_train = pd.concat([df_train, df_train_])
    df_test1 = pd.concat([df_test1, df_test1_])
    df_test2 = pd.concat([df_test2, df_test2_])


In [76]:
print("Training: {} (fraction = {})".format(df_train.shape, round(df_train.shape[0] / df_full.shape[0], 2)))
print("Test1   : {} (fraction = {})".format(df_test1.shape, round(df_test1.shape[0] / df_full.shape[0], 2)))
print("Test2   : {} (fraction = {})".format(df_test2.shape, round(df_test2.shape[0] / df_full.shape[0], 2)))

Training: (9991, 29) (fraction = 0.7)
Test1   : (2160, 29) (fraction = 0.15)
Test2   : (2165, 29) (fraction = 0.15)


In [79]:
# pickle.dump(df_train, open("2021.04.01_train_dataset","wb"), protocol=3)
# pickle.dump(df_test1, open("2021.04.01_test1_dataset","wb"), protocol=3)
# pickle.dump(df_test2, open("2021.04.01_test2_dataset","wb"), protocol=3)

# appendix

In [78]:
from collections import Counter

def find_duplicates(df):
    l1 = df.da_number#["A", "A", "B"]
    l2 = df.da_number.unique()#["A", "B"]
    c1 = Counter(l1)
    c2 = Counter(l2)

    diff = c1-c2
    duplicate_list = list(diff.elements())
#     print (len(list(diff.elements())))
#     print (list(diff.elements()))
    return duplicate_list


train_duplicate_list = find_duplicates(df_train)
test1_duplicate_list = find_duplicates(df_test1)
test2_duplicate_list = find_duplicates(df_test2)

In [53]:
df_full.shape[0], len(df_full.da_number.unique())

(14316, 14316)

In [54]:
df_train.shape[0], len(df_train.da_number.unique())

(9991, 9991)

In [55]:
df_test1.shape[0], len(df_test1.da_number.unique())

(2160, 2160)

In [56]:
df_test2.shape[0], len(df_test2.da_number.unique())

(2165, 2165)